# Loading the data into the DB, using CSV Parsers (2/3).

Following the concepts on the different [means](/HowToGuides/start/Ingestion) of ingesting the data, we will build an example using the CSV Parser method in this notebook.

We will use the [Cookbook dataset](https://docs.google.com/spreadsheets/d/1G1HPG3Dxx5W39OD6b74wMHvWupD7N-DLUbV7tD5owx8/edit?usp=sharing) to have the data persisted onto ApertureDB instance.

We convert the first 3 sheets on the source into 3 CSV files which are compatible with the CSV Parsers in the SDK. For a comprehensive understanding of the functions of the various CSV Parsers, refer to this [page](/category/csv-parsers).


## Data Models

A popular way to define the schema in python is using pydantic, and we shall use the same to create the associations of our Cookbook.


In [8]:
from typing import List, Optional
from aperturedb.DataModels import  ImageDataModel, IdentityDataModel

class Ingredient(IdentityDataModel):
    Name: str
    other_names: Optional[str] = ""
    macronutrient: Optional[str] = ""
    micronutrient: Optional[str] = ""
    subgroup: Optional[str] = ""
    category: Optional[str] = ""

class Dish(ImageDataModel):
    contributor: str
    name: str
    location: str
    caption: str
    recipe_url: str
    dish_id: int
    ingredients: List[Ingredient]


### Create objects of these classes.

We will provision the data using a json file, prepared from [script](https://github.com/aperture-data/Cookbook/blob/main/scripts/convert_ingredients_adb_csv.py)

Example lines from the json file:

:::note title="Samples from the dishes.adb.csv"
```
url,id,contributor,name,type,location,cuisine,caption,Recipe URL,constraint_id
https://raw.githubusercontent.com/aperture-data/Cookbook/refs/heads/main/images/001_Large.jpeg,1,gautam,rajma chawal,main dish,NJ,Indian,Beans with rice,https://www.tarladalal.com/rajma-chawal-punjabi-rajma-chawal-4951r,1
https://raw.githubusercontent.com/aperture-data/Cookbook/refs/heads/main/images/002_Large.jpeg,2,gautam,paneer bhurji,main dish,NJ,Indian,"Scrambled cottage cheese with finely chopped onion, bell pepper and tomatoes",https://www.indianhealthyrecipes.com/paneer-bhurji-recipe/,2
```
:::

:::note title="Samples from ingredients.adb.csv"
```
EntityClass,UUID,Name,other_names,category,subgroup,macronutrient,micronutrient,constraint_UUID
Ingredient,8ccd94efe6ac436f8c9f4b180677344a,all-purpose flour,maida,vegetarian,refined grains,carbohydrates,,8ccd94efe6ac436f8c9f4b180677344a
Ingredient,3d1ef186f6c14e61a67b01f2abfef6c4,apple,,vegetarian,fruit,carbohydrates,,3d1ef186f6c14e61a67b01f2abfef6c4

```
:::

:::note title="Samples from dish_ingredients.adb.csv"
```
ConnectionClass,_Image@id,Ingredient@UUID
HasIngredient,1,328ade6ff21244bd92332704ef72bda9
HasIngredient,1,45663216d9ea4262a608b9067adc5d1f

```
:::

## Ingesting using the CSV Parsers.

Let's ingest the same data. We shall pre process the csvs to be compatible to aperuredb's input CSV format.
The generated schma for the data will be still the same.

In [7]:
from aperturedb.ImageDataCSV import ImageDataCSV
from aperturedb.EntityDataCSV import EntityDataCSV
from aperturedb.ConnectionDataCSV import ConnectionDataCSV
from aperturedb.CommonLibrary import create_connector, execute_query

dishes_objects = ImageDataCSV("dishes.adb.csv")
ingredients_objects = EntityDataCSV("ingredients.adb.csv")
connection_objects = ConnectionDataCSV("dish_ingredients.adb.csv")


client = create_connector()

for objects in [dishes_objects, ingredients_objects, connection_objects]:
    for query, blobs in objects:
        result, response, output_blobs = execute_query(client, query, blobs)

        if result != 0:
            print(response, query)
            break